# làm sạch dữ liệu

#### Import thư viện 

In [2]:
import pandas as pd
import numpy as np
import re
import datetime
import json
import ast
from pprint import pprint

### Làm sạch dữ liệu chính (sieuanhhungMarvel.csv)

#### Đọc dữ liệu thô thư file sieuanhhungMarvel.csv

In [8]:
raw_df = pd.read_csv('Data/sieuanhhungMarvel.csv')

In [38]:
raw_df.head()

,post_id,text,time,images_low,like,love,haha,sad,wow,care,angry,reaction_count,shares,comment,comments_full,year,month,day
0,734518898719448,Các bạn nam chắc ai cũng muốn học chiêu pause ...,2023-11-18 22:40:26,['https://scontent.fhan14-3.fna.fbcdn.net/v/t3...,1983,18,801,8,1,6,0,2817,21,101,"[{'comment_id': '302120192733098', 'comment_ur...",2023,11,18
1,734285772076094,Nếu như Pedro một mình cân tất cả các vai ở Fa...,2023-11-18 12:51:48,['https://scontent.fhan14-2.fna.fbcdn.net/v/t3...,2014,19,1667,21,4,7,1,3733,13,94,"[{'comment_id': '361135779633240', 'comment_ur...",2023,11,18
2,734234215414583,Sau bom tấn Spider-Man 2 thì dự án tiếp theo c...,2023-11-18 10:21:01,['https://scontent.fhan14-2.fna.fbcdn.net/v/t3...,1543,45,275,5,8,5,1,1882,9,51,"[{'comment_id': '1012246983374962', 'comment_u...",2023,11,18
3,734194055418599,Infinity Magic 🔥,2023-11-18 08:30:34,['https://scontent.fhan14-1.fna.fbcdn.net/v/t3...,3997,638,18,4,14,26,1,4698,34,60,"[{'comment_id': '1842669876147189', 'comment_u...",2023,11,18
4,733990162105655,Nam diễn viên Steven Yeun được xác nhận từ nhi...,2023-11-17 23:17:50,['https://scontent.fhan14-3.fna.fbcdn.net/v/t3...,732,56,171,1,2,3,1,966,5,56,"[{'comment_id': '1410796539783216', 'comment_u...",2023,11,17


#### Làm sạch dữ liệu hình ảnh từ dạng string sang dạng list

In [10]:
for i in range(len(raw_df.images_low)):
    text = raw_df.images_low.iloc[i]
    raw_df.iat[i, raw_df.columns.get_loc('images_low')] = ast.literal_eval(text)

#### Thêm cột image_count

In [12]:
image_count = []
for i in range(len(raw_df)):
    image_count.append(len(raw_df.iloc[i].images_low))

raw_df['images_count'] = image_count

raw_df.to_csv('Data/sieuanhhungMarvel.csv', index=False)

#### Thêm cột Year, Month, Date

In [40]:
# Convert 'time' to datetime format
raw_df['time'] = pd.to_datetime(raw_df['time'])

# Extract year, month, and day into separate columns
raw_df['year'] = raw_df['time'].dt.year
raw_df['month'] = raw_df['time'].dt.month
raw_df['day'] = raw_df['time'].dt.day

raw_df.to_csv('Data/sieuanhhungMarvel.csv', index=False)

In [41]:
raw_df.head()

,post_id,text,time,images_low,like,love,haha,sad,wow,care,angry,reaction_count,shares,comment,comments_full,year,month,day
0,734518898719448,Các bạn nam chắc ai cũng muốn học chiêu pause ...,2023-11-18 22:40:26,[https://scontent.fhan14-3.fna.fbcdn.net/v/t39...,1983,18,801,8,1,6,0,2817,21,101,"[{'comment_id': '302120192733098', 'comment_ur...",2023,11,18
1,734285772076094,Nếu như Pedro một mình cân tất cả các vai ở Fa...,2023-11-18 12:51:48,[https://scontent.fhan14-2.fna.fbcdn.net/v/t39...,2014,19,1667,21,4,7,1,3733,13,94,"[{'comment_id': '361135779633240', 'comment_ur...",2023,11,18
2,734234215414583,Sau bom tấn Spider-Man 2 thì dự án tiếp theo c...,2023-11-18 10:21:01,[https://scontent.fhan14-2.fna.fbcdn.net/v/t39...,1543,45,275,5,8,5,1,1882,9,51,"[{'comment_id': '1012246983374962', 'comment_u...",2023,11,18
3,734194055418599,Infinity Magic 🔥,2023-11-18 08:30:34,[https://scontent.fhan14-1.fna.fbcdn.net/v/t39...,3997,638,18,4,14,26,1,4698,34,60,"[{'comment_id': '1842669876147189', 'comment_u...",2023,11,18
4,733990162105655,Nam diễn viên Steven Yeun được xác nhận từ nhi...,2023-11-17 23:17:50,[https://scontent.fhan14-3.fna.fbcdn.net/v/t39...,732,56,171,1,2,3,1,966,5,56,"[{'comment_id': '1410796539783216', 'comment_u...",2023,11,17


#### Làm sạch comments_full (id commenters, comment time, reaction count, reactions type)

In [42]:
text = raw_df.comments_full.iloc[0]

In [43]:
pprint(text)

("[{'comment_id': '302120192733098', 'comment_url': "
 "'https://facebook.com/302120192733098', 'commenter_id': '100016600359265', "
 "'commenter_url': "
 "'https://facebook.com/dysphotica?eav=AfaSYrxe_s6RMF6KM39qlCELCzG3M9qVe7fP7E74jI1fscnI0X4dhlb0Im7zuvQ0fXE&fref=nf&rc=p&refid=52&__tn__=R&paipv=0', "
 "'commenter_name': 'Viet Hung', 'commenter_meta': None, 'comment_text': 'tôi "
 "có ý này...', 'comment_time': datetime.datetime(2023, 11, 18, 0, 0), "
 "'comment_image': "
 "'https://scontent-iad3-1.xx.fbcdn.net/m1/v/t6/An_F9bJG7govfshSMBkvcRLcxT0jmiXVYKtr7lgH5AHgUrjjpZ1OD0xyxXYgf7arc0lWgCdrR_KN4Mg7RSN3Gm3W6Gg03N1tQ-ZXzVvFJ_KvvB4.png?ccb=10-5&oh=00_AfDqLHGOqOIMLAGbcyjut-IQT2S7S9rL5VBZKTDC3k3dSQ&oe=6585F867&_nc_sid=7da55a', "
 "'comment_reactors': [{'name': 'Đặng Nhật Minh', 'link': "
 "'https://facebook.com/profile.php?id=100093996697870&eav=AfacZo1j6mSpvtahdC9rtAlyr2bdAXMMvW3dHXC9ASDAeXN-gRwVxHU5x_dYOVdGOkA&fref=pb&paipv=0', "
 "'type': 'haha'}, {'name': 'Nguyễn Duy', 'link': "
 "'htt

In [44]:
start_marker = "commenter_id"
end_marker = "replies"

start_index = text.find(start_marker)
end_index = text.find(end_marker, start_index)

if start_index != -1 and end_index != -1:
    result = text[start_index : end_index].strip()
    print(result)
else:
    print("Markers not found.")


commenter_id': '100016600359265', 'commenter_url': 'https://facebook.com/dysphotica?eav=AfaSYrxe_s6RMF6KM39qlCELCzG3M9qVe7fP7E74jI1fscnI0X4dhlb0Im7zuvQ0fXE&fref=nf&rc=p&refid=52&__tn__=R&paipv=0', 'commenter_name': 'Viet Hung', 'commenter_meta': None, 'comment_text': 'tôi có ý này...', 'comment_time': datetime.datetime(2023, 11, 18, 0, 0), 'comment_image': 'https://scontent-iad3-1.xx.fbcdn.net/m1/v/t6/An_F9bJG7govfshSMBkvcRLcxT0jmiXVYKtr7lgH5AHgUrjjpZ1OD0xyxXYgf7arc0lWgCdrR_KN4Mg7RSN3Gm3W6Gg03N1tQ-ZXzVvFJ_KvvB4.png?ccb=10-5&oh=00_AfDqLHGOqOIMLAGbcyjut-IQT2S7S9rL5VBZKTDC3k3dSQ&oe=6585F867&_nc_sid=7da55a', 'comment_reactors': [{'name': 'Đặng Nhật Minh', 'link': 'https://facebook.com/profile.php?id=100093996697870&eav=AfacZo1j6mSpvtahdC9rtAlyr2bdAXMMvW3dHXC9ASDAeXN-gRwVxHU5x_dYOVdGOkA&fref=pb&paipv=0', 'type': 'haha'}, {'name': 'Nguyễn Duy', 'link': 'https://facebook.com/duynm07?eav=AfbzJgdc9IcEMi7doYi4nE31UIAubkizkNNE5HGYw7utdWis0ervsiBVtyIgblF-tCM&fref=pb&paipv=0', 'type': 'haha'}, {'na

In [47]:
start_marker = "commenter_id"
end_marker = "comment_reaction_count"

# Tách chuỗi thành các phần dựa trên end_marker
sections = text.split(end_marker)

count = 0

for section in sections:
    count += 1
    # Tìm vị trí của start_marker trong từng phần
    start_index = section.find(start_marker)
    
    # Nếu start_marker được tìm thấy trong phần
    if start_index != -1:
        # Trích xuất từ start_marker đến cuối phần
        result = section[start_index:].strip()
        print(result)

print(count)

commenter_id': '100016600359265', 'commenter_url': 'https://facebook.com/dysphotica?eav=AfaSYrxe_s6RMF6KM39qlCELCzG3M9qVe7fP7E74jI1fscnI0X4dhlb0Im7zuvQ0fXE&fref=nf&rc=p&refid=52&__tn__=R&paipv=0', 'commenter_name': 'Viet Hung', 'commenter_meta': None, 'comment_text': 'tôi có ý này...', 'comment_time': datetime.datetime(2023, 11, 18, 0, 0), 'comment_image': 'https://scontent-iad3-1.xx.fbcdn.net/m1/v/t6/An_F9bJG7govfshSMBkvcRLcxT0jmiXVYKtr7lgH5AHgUrjjpZ1OD0xyxXYgf7arc0lWgCdrR_KN4Mg7RSN3Gm3W6Gg03N1tQ-ZXzVvFJ_KvvB4.png?ccb=10-5&oh=00_AfDqLHGOqOIMLAGbcyjut-IQT2S7S9rL5VBZKTDC3k3dSQ&oe=6585F867&_nc_sid=7da55a', 'comment_reactors': [{'name': 'Đặng Nhật Minh', 'link': 'https://facebook.com/profile.php?id=100093996697870&eav=AfacZo1j6mSpvtahdC9rtAlyr2bdAXMMvW3dHXC9ASDAeXN-gRwVxHU5x_dYOVdGOkA&fref=pb&paipv=0', 'type': 'haha'}, {'name': 'Nguyễn Duy', 'link': 'https://facebook.com/duynm07?eav=AfbzJgdc9IcEMi7doYi4nE31UIAubkizkNNE5HGYw7utdWis0ervsiBVtyIgblF-tCM&fref=pb&paipv=0', 'type': 'haha'}, {'na

### Làm sạch file reactors (reactors lấy từ post có nhiều lượt thả reaction nhất)

In [3]:
reactor_df = pd.read_csv('Data/reactos.csv')
reactor_df

,name,link,type
0,Phan Thanh Tùng,https://facebook.com/profile.php?id=1000956057...,haha
1,Diem Quynh,https://facebook.com/profile.php?id=1000956051...,haha
2,Nguyễn Thị Xuân Trâm,https://facebook.com/profile.php?id=1000955797...,haha
3,Woo Kyu,https://facebook.com/profile.php?id=1000955591...,like
4,Area Luna,https://facebook.com/profile.php?id=1000955131...,love
...,...,...,...
4453,Minh Đức,https://facebook.com/profile.php?id=1000505264...,NaN
4454,Nguyễn Mai Anh,https://facebook.com/profile.php?id=1000505244...,NaN
4455,Vy Hà,https://facebook.com/profile.php?id=1000505154...,NaN
4456,Lê Hồ Cẩm Phúc,https://facebook.com/phucc.caam0619?eav=AfYSIY...,NaN


#### Xóa cột type

In [5]:
reactor_df = reactor_df.drop('type', axis=1)
reactor_df

,name,link
0,Phan Thanh Tùng,https://facebook.com/profile.php?id=1000956057...
1,Diem Quynh,https://facebook.com/profile.php?id=1000956051...
2,Nguyễn Thị Xuân Trâm,https://facebook.com/profile.php?id=1000955797...
3,Woo Kyu,https://facebook.com/profile.php?id=1000955591...
4,Area Luna,https://facebook.com/profile.php?id=1000955131...
...,...,...
4453,Minh Đức,https://facebook.com/profile.php?id=1000505264...
4454,Nguyễn Mai Anh,https://facebook.com/profile.php?id=1000505244...
4455,Vy Hà,https://facebook.com/profile.php?id=1000505154...
4456,Lê Hồ Cẩm Phúc,https://facebook.com/phucc.caam0619?eav=AfYSIY...


In [8]:
reactor_df.link.iloc[0]

'https://facebook.com/profile.php?id=100095605751322&eav=AfZM_Cg-nI7lLJFb2hOtIlb10miqZyQBrFuVd4c_Kj2Vy1YdOM7ezAxIvROhCJaNESA&fref=pb&paipv=0'

#### Lấy id của user từ link

In [12]:
def get_user_id(link):
    if "profile.php?id" in link:
        id = link.split('profile.php?id=')[1].split('&eav')[0]
    else:
        id = link.split('facebook.com/')[1].split('?eav')[0]
    return id

In [13]:
ids = []
for link in reactor_df.link:
    id = get_user_id(link)
    print(id)
    ids.append(id)

100095605751322
100095605181131
100095579735068
100095559191414
100095513197010
100095501824916
100095452743899
100095434171599
100095429194330
100095421480417
100095417109212
100095412789379
100095411910334
100095387584780
100095367398916
100095367381775
100095353672070
100095345148179
100095324197090
100095307580713
100095277850673
100095249123118
100095248173975
100095224782820
100095221985352
100095210266848
100095204133715
100095194924227
100095189826811
100095182105653
100095170652873
100095164163071
100095145227481
100095132095783
100095114949158
100095113620827
100095100465000
100095098954012
100095095773236
100095069095389
100095065742914
100095052920364
100095049036600
100095047938968
100095017343802
100095000869862
100094986514647
100095204943710
100095375055911
100095005871497
100094978835648
100094971516962
100094957363380
100094930231790
100094921938010
100094911987223
100094909831476
100094885342940
100094881338401
100094786655811
100094777704612
100094770826236
10009477

#### Thêm cột user id

In [15]:
reactor_df['user_id'] = ids

#### Xóa cột link

In [17]:
reactor_df = reactor_df.drop('link', axis=1)
reactor_df

,name,user_id
0,Phan Thanh Tùng,100095605751322
1,Diem Quynh,100095605181131
2,Nguyễn Thị Xuân Trâm,100095579735068
3,Woo Kyu,100095559191414
4,Area Luna,100095513197010
...,...,...
4453,Minh Đức,100050526411681
4454,Nguyễn Mai Anh,100050524402263
4455,Vy Hà,100050515492422
4456,Lê Hồ Cẩm Phúc,phucc.caam0619


#### Lưu dữ liệu đã làm sạch vào file

In [ ]:
reactor_df.to_csv('Data/reactos.csv', index=False)